In [ ]:
import sys
sys.path.append('../')

import logging
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from marmopose.version import __version__ as marmopose_version
from marmopose.config import Config
from marmopose.calibration.calibration import Calibrator
from marmopose.processing.prediction import Predictor
from marmopose.visualization.display_2d import Visualizer2D
from marmopose.visualization.display_3d import Visualizer3D
from marmopose.processing.triangulation import Reconstructor3D

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
logger = logging.getLogger(__name__)

logger.info(f'MarmoPose version: {marmopose_version}')

In [ ]:
config_path = '../configs/default.yaml'

config = Config(
    config_path=config_path,

    # By default, 'detection_model_family' will identify 4 types of marmosets: {'white_head_marmoset':0, 'blue_head_marmoset':1, 'green_head_marmoset':2, 'red_head_marmoset':3}
    # In this example, we aim to track a subset of the instances: 'blue_head_marmoset' and 'green_head_marmoset'
    # We first need to set 'n_tracks' to the number of marmosets we want to track (here is 2), then specify the label_mapping form 'label_id' to 'track_id'
    # The model's output label_id of 'blue_head_marmoset' is 1, 'green_head_marmoset' is 2
    # Here we map their track_id to 1 and 0 respectively (where track_id should be in 'range(n_tracks)')
    n_tracks=2, 
    label_mapping = {
        1: 1,
        2: 0
    },
    project='../demos/family_subset',

    det_model='../models/detection_model_family',
    # Use default pose model in the config
    
    do_optimize=True 
)

In [ ]:
# Run 2D prediction
predictor = Predictor(config, batch_size=4)
predictor.predict()

In [ ]:
# Run 3D triangulation
reconstructor_3d = Reconstructor3D(config)
reconstructor_3d.triangulate()

In [ ]:
# Visualize 2D results if needed
visualizer_2d = Visualizer2D(config)
visualizer_2d.generate_videos_2d()

In [ ]:
# Visualize 3D results if needed
visualizer_3d = Visualizer3D(config)

# Visualize the original 3D poses
visualizer_3d.generate_video_3d(source='original') # Generate 3D video
visualizer_3d.generate_video_combined(source='original') # Combine 2D and 3D video for convenience

# Visualize the smoothed & optimized 3D poses (require `do_optimize=True` in the config)
visualizer_3d.generate_video_3d(source='optimized')
visualizer_3d.generate_video_combined(source='optimized')